# Summary

## Documents

- A Document is the unix of text that the LLM processes in order to answer queries. 
- A Document is not necessarly the whole file: files are typically chunked. Documents usually contain metadata including the name of the original document, the page from where the part of the document was extracted, author e.t.c.
- Full texts are usually split into document chunks to optimise LLm output. Larger chunks = more stuff in context = more risk of hallunication. 

## Embeddings

- LLMs store text as vectors in a high-dimensional space
- In this space, the position of each point (embedding) reflects the meaning of its corresponding text.
Just as similar words might be close to each other in a thesaurus, similar concepts end up close to each other in this embedding space.
- In simpler words: LLMs store text in "space". The way that text is store relative to each other captures the relationship between the text. So text close to each other might have similar meaning. Text at a higher vertical axis might describe parent chld relationship. 
 
## Vector Stores

- A vector store is a specialised database for storing and querying embeddings. It supports standard operations like create, read, update, and delete (CRUD).
- For retrieval, it allows searching for semantically similar text by comparing embedding vectors using similarity metrics such as cosine similarity.


## Tool Calling
- Tool calling enables an LLM to interact with systems e.g. calling an API or querying a database.
- When interacting with external tools, the request and response typically needs to confirm to a schema e.g. API Request Payload, SQL query structure.

## Structured Output

- 

## Few-shot Prompting
- Few-shot prompting is a technique used with large language models (LLMs) where you provide a small number of examples (typically 1–5) within the prompt to show the model how to perform a task
- Importantly, the examples should include "negatives" so that the LLM understands how to handle such cases.
- **Different chat model providers impose different requirements for valid message sequences**

## Chatbots
- By default, an LLM does not retain the context from previous invocations. For example, if you tell an LLM your name in one invocation, it will not "remember" your name in the subsequent invocations
- For the LLM to remember the name, the chat history has to be sent with each invocation
- Tools like LangGraph enable this.

## LangGraph
- **LangGraph** is an open-source library from the creators of LangChain that is used to build A stateful, multi-step agent/workflow applications with reliable execution and persistence.
- Importantly, Human/Agent interactions are modeled as nodes in a graph which makes the orchestration between humans and agents visible and easy to debug.

#### Nodes
- A node is a single action: e.g., call an LLM, run a tool, query a database, or invoke custom code.
- Nodes can represent different actors (LLMs, tools, humans).
- In case of a human, the graph encodes when to pause for human input, how to resume, and how actors exchange state.

#### Edges
- Edges connect nodes and decide what runs next.
- They can be unconditional, conditional (branching), looping, or fan-out/fan-in (parallel paths and joins).
- Graph branches can run in parallel

#### Graph
- A workflow is the directed graph of nodes and edges.
- This makes the system explicit and debuggable: you can see the exact path the execution took.

#### State
- The workflow carries a state object (typically a structured dict/schema) that persists across nodes.
- Nodes read from state and propose updates to state rather than mutating it in place
- After each node runs, LangGraph materialises a new state version.
- This **immutability per step** yields reproducibility, diff-ability, and clear audit trails.

#### Checkpoints
- Each step can be checkpointed (state snapshot + metadata).
- Checkpoints and state can be stored in memory or external stores (DBs, object storage).

- With checkpoints, you can retry or reroute without losing prior work.
- This enables resume after failure/timeouts, deterministic replay, and precise debugging from any point.

#### Thread
- A thread is one concrete run of the graph (e.g., a single user session).
- Each thread has its own state history and checkpoints, isolating concurrent users/sessions cleanly.

![](./docs/langgraph-checkpoints.jpg)

```python
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"), # state['messages'] will be inserted in this placeholder.
    ]
)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages] # state stores messages
    language: str # messages contain a variable 'language' whose value also needs to be stored with the mesages


workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
````